In [1]:
import classes
import functies
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
###bereken de fouten op de verdunningsreeksen
Cstock, Vstock, Vadd, Cnew = sp.symbols('Cstock Vstock Vadd Cnew')
Cnewvgl = classes.vergelijking(Cstock*Vstock/(Vadd+Vstock), [Cstock, Vstock, Vadd])

cstock = functies.vector_to_datapunt([60e-6, 0.5e-6, 'N'], Cstock) #M
#verdunning 1
vs1 = [3e-3, 0.1e-3, 'U'] #l, gemeten met gegradueerd pipet
va1 = [57e-3, 0.05e-3, 'U'] #l, gemeten met buret
#verdunning 2
vs2 = [10e-3, 0.02e-3, 'U'] #l, gemeten met volumetrisch pipet
va2 = [90e-3, 0.1e-3 , 'U'] #l, gemeten met maatkolf
#verdunning 3
vs3 = [6e-3, 0.1e-3, 'U'] #l, gemeten met gegradueerd pipet
va3 = [34e-3, 0.05e-3, 'U'] #l, gemeten met buret
#verdunning 4
vs4 = [20e-3, 0.02e-3, 'U'] #l, gemeten met volumetrisch pipet
va4 = [80e-3, 0.1e-3 , 'U'] #l, gemeten met maatkolf

stockvolumes_matrix = [vs1, vs2, vs3, vs4]
toegevoegde_volumes_matrix = [va1, va2, va3, va4]

stockvolumes_datapunten = functies.matrix_to_datapunten(stockvolumes_matrix, Vstock)
toegevoegde_volumes_datapunten = functies.matrix_to_datapunten(toegevoegde_volumes_matrix, Vadd)
parameters = [Cstock, Vstock, Vadd]
metingen = [[cstock, stockvolumes_datapunten[i],toegevoegde_volumes_datapunten[i]] for i in range(len(stockvolumes_datapunten))]
print(metingen[0])
cnew = functies.multiple_analysis(Cnewvgl,metingen,Cnew)
print(cnew)

[datapunt: Cstock = [6e-05, 5e-07, 'N'], datapunt: Vstock = [0.003, 0.0001, 'U'], datapunt: Vadd = [0.057, 5e-05, 'U']]
[datapunt: Cnew = [3.00000000000000e-6, 3.16696544643116e-12, 'Normaal']
 datapunt: Cnew = [6.00000000000000e-6, 2.14030371676545e-13, 'Normaal']
 datapunt: Cnew = [9.00000000000000e-6, 4.25429929600634e-12, 'Normaal']
 datapunt: Cnew = [1.20000000000000e-5, 4.08161732650184e-13, 'Normaal']]


In [3]:
def lineair(c, param):
    epsiloneff = param[0]
    return epsiloneff*c
def initial_vals(x,y):
    return [1]
concentraties = np.array([3e-6, 6e-6, 9e-6, 12e-6]) #molair
absorbanties = np.array([0.268, 0.557, 0.803, 1.022]) #dimensieloos

fouten = 0.02*absorbanties

absorbantie_onbekend = [1.690, 0.02*1.690, 'U']

In [6]:
#fit het boeltje
parameter = ['epsilon_effectief']
epsilon_fitwaarde = functies.fit(parameter, lineair, initial_vals, concentraties, absorbanties, y_err = fouten)[0]

A, epsilon_eff = sp.symbols('A epsilon_eff')
concentratie_voorschrift = classes.vergelijking(A/epsilon_eff, [A, epsilon_eff])
absorbantie_onbekend = [1.690, 0.02*1.690, 'U']
absorbantie_onbekend = functies.vector_to_datapunt(absorbantie_onbekend, A)
epsilon_fitwaarde = functies.vector_to_datapunt(outp[0], epsilon_eff)

concentratie_onbekend = functies.data_analyse(concentratie_voorschrift, [A, epsilon_eff], [absorbantie_onbekend, epsilon_fitwaarde])
print(concentratie_onbekend)


Raw output
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 9.376751713983147
        x: [ 8.897e+04]
      nit: 3
      jac: [ 3.576e-07]
 hess_inv: [[ 3.962e+05]]
     nfev: 24
     njev: 12
[[88087.05582700793, 89857.4694216154]]
epsilon_effectief heeft als waarde: 88972 + 885.27 - 885.14 met 68% betrouwbaarheidsinterval: [88087, 89857] 
De p-waarde voor de hypothese test dat het model zinvol is, wordt gegeven door: 0.024679
De gereduceerde chi^2 waarde is: 3.1256


AttributeError: 'Symbol' object has no attribute 'variance'

In [5]:
fig, ax = plt.subplots(1,1, figsize = (8,8))

#de gekende punten en de fit
ax.errorbar(concentraties, absorbanties, yerr = y_fouten, linestyle = ' ', color = 'black', label = 'gemeten')
concentratie_linspace = np.linspace(0,20e-6)
ax.plot(concentratie_linspace, lineair(concentratie_linspace, [epsilon_fitwaarde.get_val()]), label = 'model')

#het onbekende punt
ax.errorbar(concentratie_onbekend.get_val(),absorbantie_onbekend.get_val(), yerr = absorbantie_onbekend.get_fout(), xerr = concentratie_onbekend.get_fout(), label = 'onbekende concentratie')


ax.set_xlabel('concentratie [M]')
ax.set_ylabel('absorbantie [dimensieloos]')
ax.set_title('De concentraties versus de absorbanties van erythrocine-B')
ax.legend()
plt.tight_layout();plt.show()

NameError: name 'plt' is not defined